### This will be the continuously tested version. Testing mostly done in Spyder, then the correct code is uploaded here.

In [1]:
import tkinter as tk
from tkinter import simpledialog, scrolledtext
from tkinter import messagebox #this is for pop up when illegal characters was input
import time
import csv
import os

def save_data(data, filepath):
    headers = ["ID", "TotalKeystrokes1", "TotalBackspaces1", "TimeElapsed1", "InputText1", "StressRating1", 
               "TotalKeystrokes2", "TotalBackspaces2", "TimeElapsed2", "InputText2", "StressRating2",
               "TotalKeystrokes3", "TotalBackspaces3", "TimeElapsed3", "InputText3", "StressRating3",
               "TotalKeystrokes4", "TotalBackspaces4", "TimeElapsed4", "InputText4", "StressRating4"]
    file_exists = os.path.isfile(filepath)
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

def open_summary_window(data, root, round_index):
    summary_window = tk.Toplevel(root)
    summary_window.title(f"Summary of Your Input - Round {round_index}")
    summary_window.geometry("300x200")
    text_display = (f"ID: {data['ID']}\n"
                    f"Total Keystrokes: {data[f'TotalKeystrokes{round_index}']}\n"
                    f"Total Backspaces: {data[f'TotalBackspaces{round_index}']}\n"
                    f"Time Elapsed: {data[f'TimeElapsed{round_index}']} seconds\n"
                    f"Stress Rating: {data[f'StressRating{round_index}']}")
    summary_label = tk.Label(summary_window, text=text_display, justify="left", wraplength=280)
    summary_label.pack(pady=20)

    if round_index == "4":
        save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")
        print("Data saved to filepath")
        close_button = tk.Button(summary_window, text="Save and Exit", command=lambda: end_and_save(root))
        print("Exit program")
        close_button.pack(side=tk.LEFT, padx=10, pady=5)
        return_button = tk.Button(summary_window, text="Return to Beginning", command=lambda: return_to_start(summary_window, root))
        print("This user data saved. Ready for next user")
        return_button.pack(side=tk.RIGHT, padx=10, pady=5)
    else:
        next_button = tk.Button(summary_window, text="Next", command=lambda: [summary_window.destroy(), open_reply_window(root, data["ID"], str(int(round_index) + 1), data)])
        next_button.pack(pady=5)

 # Initialize the backspace counter
backspace_count = 0

def open_reply_window(root, user_id, round_index, data):
    reply_window = tk.Toplevel(root)
    reply_window.title(f"Reply to Supervisor - Round {round_index}")
    reply_window.geometry("600x400")
    email_message = {
        #Round 1 Chinese neutral, round 2 Chinese negative, round 3 english neutral, round 4 english negative
        "1": "你好,\n\n公司刚收到提醒，有一个重要报告截止日期在下一个月的7号。 "
             "这个项目需要你的组尽量完成，如果需要加入临时人员，请与我联系。 "
             "就目前你们的进度来看，项目在月初结束应该是没有问题的！ "
             "另外，希望你能代表你们组在每个星期的工作进展会上汇报一下工作完成进度。\n\n"
             "有问题请及时与我沟通，加油！\n--项目组组长",
        "2": "Hello,\n\nThe company has just received a reminder that an important progress report is "
             "due early next month on the 7th. This project would require your current team's expertise. "
             "If your teams would require extra help, please let me know. "
             "Based on the current progress, I believe the project can be wrapped up early next month "
             "without a problem! I would like for you to give a minor progress report during the "
             "weekly company meetings. Please let me know if you have any questions. Good luck!"
             "\n\nBest regards,\nYour Supervisor",
        "3": "你好,\n\n我注意到你之前请假，现在可能需要时间来重新调整状态。 "
             "然而，我也想提醒你，我们的项目进展需要你的组里及时参与和努力，需要加紧完成。 "
             "就目前来看，你之前请假的理由不足以解释你现在仍未完成项目的情况。 "
             "我希望你能赶紧进入状态投入工作，确保我们组能够按时完成项目。\n\n"
             "有问题请及时与我沟通。\n--项目组组长",
        "4": "Hello,\n\nI noticed that you previously took leave and may now need time to re-adjust. "
             "I also want to remind you that the progress of our project requires your timely "
             "participation and effort, and we might need you to expedite your work progress. "
             "Based on the current situation, the reasons for your previous leave are insufficient "
             "to explain why you have not yet completed the project. "
             "Please let me know if you have any questions, or encounter any problems."
             "\n\nBest regards,\nYour Supervisor"            
        
    }[round_index]
    message_label = tk.Label(reply_window, text=email_message, justify="left", wraplength=580)
    message_label.pack(pady=(10, 20))
    text_box = scrolledtext.ScrolledText(reply_window, width=50, height=10)
    text_box.pack()
    timer_label = tk.Label(reply_window, text="00:00", font=('Helvetica', 14))
    timer_label.pack(anchor='ne')
    start_time = time.time()
    
    def update_timer():
        if not getattr(reply_window, 'stop_timer', False):
            elapsed_time = time.time() - start_time
            minutes, seconds = divmod(int(elapsed_time), 60)
            timer_label.config(text=f"{minutes:02}:{seconds:02}")
            reply_window.after(1000, update_timer)
    update_timer()
    
    def on_backspace(event):
        global backspace_count
        backspace_count += 1

    def handle_send(data):
        global backspace_count
        reply_window.stop_timer = True
        characters = len(text_box.get("1.0", "end-1c"))
        if characters == 0:
            print("User attempted to submit empty box")
            messagebox.showwarning("Warning", "Please input some response before sending.")
            return
        input_text = text_box.get("1.0", "end-1c")
        elapsed_time = time.time() - start_time
        data[f"TotalKeystrokes{round_index}"] = characters
        data[f"TotalBackspaces{round_index}"] = backspace_count
        data[f"TimeElapsed{round_index}"]     = int(elapsed_time)
        data[f"InputText{round_index}"]       = input_text
        data[f"StressRating{round_index}"]    = 0 # Initialized for update in stress rating
        #save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")  #I deleted this this was called too many times
        reply_window.destroy()
        open_stress_window(data, root, round_index)
        backspace_count = 0 # Reset backspace_count for the next session if needed
        
    # Bind the backspace key event to the text box
    text_box.bind("<BackSpace>", on_backspace)
    
    send_button = tk.Button(reply_window, text="Send", command=lambda: handle_send(data))
    send_button.pack(pady=10)
    reply_window.protocol("WM_DELETE_WINDOW", lambda: handle_send(data))  # Handle window close as send

def end_and_save(root):
    root.quit()
    print("Root quit executed")
    root.destroy()
    print("All windows closed and program terminated")

def open_stress_window(data, root, round_index):
    stress_window = tk.Toplevel(root)
    stress_window.title("Stress Level Rating")
    stress_window.geometry("400x200")

    instruction_text = "Rate the level of stress experienced (0-10, decimals allowed):"
    instruction_label = tk.Label(stress_window, text=instruction_text, wraplength=380)
    instruction_label.pack(pady=20)

    stress_entry = tk.Entry(stress_window)
    stress_entry.pack(pady=10)

    submit_button = tk.Button(stress_window, text="Submit", command=lambda: save_stress_and_proceed(data, stress_entry.get(), stress_window, root, round_index))
    submit_button.pack(pady=10)

def save_stress_and_proceed(data, stress_rating, stress_window, root, round_index):
    if not stress_rating:  # Check if the input is empty
        print("User attempted to enter empty box")
        messagebox.showerror("Error", "Please enter a number!", parent=stress_window)
        return
    try:
        stress_rating = float(stress_rating)  # Ensure stress_rating is converted to float
        if 0 <= stress_rating <= 10:    
            data[f'StressRating{round_index}'] = stress_rating  # Update the stress rating in data
            stress_window.destroy()  # Close the stress rating window
            open_summary_window(data, root, round_index)  # Open the summary window
        else: 
            print("Illegal input")
            messagebox.showerror("Error", "Please enter a number between 0 and 10!", parent=stress_window)
    except ValueError:
            messagebox.showerror("Error", "Invalid input. Please enter a numeric value (0-10).", parent=stress_window)
    
def return_to_start(summary_window, root):
    summary_window.destroy()
    open_instructions_window(root)

def open_instructions_window(root):
    instructions_window = tk.Toplevel(root)
    instructions_window.title("Instructions")
    instructions_window.geometry("600x400")
    instruction_text = ("You have entered your ID. Please read the following instructions carefully:\n"
                        "1. You will read two emails sent by your supervisor.\n"
                        "2. Two rounds will be in Chinese and two rounds in English.\n" #Add if you need new round for neutral (control)
                        "3. Please respond ONLY in ENGLISH.\n"
                        "4. Please do not click SEND until you are satisfied with your response.\n"
                        "5. Please do not click SAVE AND EXIT until your experimenter says so.\n")
    instructions_label = tk.Label(instructions_window, text=instruction_text, justify="left", wraplength=580)
    instructions_label.pack(pady=20)
    
    # Keep the instructions window on top until a valid ID is entered
    instructions_window.grab_set()
    
    user_id = None
    while not user_id:
        user_id = simpledialog.askstring("Input", "Enter your ID number:", parent=instructions_window)
        if user_id:  # If a user ID was entered
            data = {}
            data.setdefault("ID", user_id)
            instructions_window.destroy()  # Destroy the window as a valid ID is entered
            open_reply_window(root, user_id, "1", data)
        else:
            messagebox.showwarning("ID Required", "You must enter an ID to proceed.", parent=instructions_window)

def main():
    root = tk.Tk()
    root.withdraw()
    open_instructions_window(root)
    root.mainloop()

if __name__ == "__main__":
    main()

Data saved to filepath
Exit program
This user data saved. Ready for next user
Root quit executed
All windows closed and program terminated
